# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 8:30AM,246460,10,MTR-22-0252,"Senseonics, Incorporated",Released
1,Sep 15 2022 8:30AM,246462,10,TXMD-DR-0100-BO,"TherapeuticsMD, Inc.",Released
2,Sep 15 2022 8:30AM,246469,10,CTF0011395,"Citieffe, Inc.",Released
3,Sep 15 2022 7:55AM,246475,19,ADV80005686,"AdvaGen Pharma, Ltd",Released
4,Sep 15 2022 7:55AM,246475,19,ADV80005688,"AdvaGen Pharma, Ltd",Released
5,Sep 15 2022 7:55AM,246475,19,ADV80005689,"AdvaGen Pharma, Ltd",Released
6,Sep 15 2022 7:55AM,246475,19,ADV80005690,"AdvaGen Pharma, Ltd",Released
7,Sep 15 2022 7:55AM,246475,19,ADV80005691,"AdvaGen Pharma, Ltd",Released
8,Sep 15 2022 7:55AM,246475,19,ADV80005692,"AdvaGen Pharma, Ltd",Released
9,Sep 15 2022 7:34AM,246474,22,627458.1,"NBTY Global, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,246470,Released,1
18,246471,Released,1
19,246473,Released,1
20,246474,Executing,1
21,246475,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246470,NaN,NaN,1.0
246471,NaN,NaN,1.0
246473,NaN,NaN,1.0
246474,NaN,1.0,NaN
246475,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246358,0.0,1.0,0.0
246424,0.0,3.0,0.0
246428,0.0,0.0,1.0
246429,0.0,0.0,1.0
246435,6.0,15.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246358,0,1,0
246424,0,3,0
246428,0,0,1
246429,0,0,1
246435,6,15,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246358,0,1,0
1,246424,0,3,0
2,246428,0,0,1
3,246429,0,0,1
4,246435,6,15,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246358,,1,
1,246424,,3,
2,246428,,,1
3,246429,,,1
4,246435,6,15,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated"
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc."
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc."
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd"
9,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc."
10,Sep 15 2022 7:16AM,246473,22,"NBTY Global, Inc."
11,Sep 14 2022 6:46PM,246471,16,Sartorius Bioprocess Solutions
12,Sep 14 2022 6:45PM,246470,16,Sartorius Lab Products and Service
13,Sep 14 2022 6:01PM,246468,16,Sartorius Bioprocess Solutions
14,Sep 14 2022 4:58PM,246466,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",,,1
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",,,1
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc.",,,1
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd",,,6
4,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc.",,1,
5,Sep 15 2022 7:16AM,246473,22,"NBTY Global, Inc.",,,1
6,Sep 14 2022 6:46PM,246471,16,Sartorius Bioprocess Solutions,,,1
7,Sep 14 2022 6:45PM,246470,16,Sartorius Lab Products and Service,,,1
8,Sep 14 2022 6:01PM,246468,16,Sartorius Bioprocess Solutions,,,1
9,Sep 14 2022 4:58PM,246466,20,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1,,
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1,,
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc.",1,,
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd",6,,
4,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc.",,1,
5,Sep 15 2022 7:16AM,246473,22,"NBTY Global, Inc.",1,,
6,Sep 14 2022 6:46PM,246471,16,Sartorius Bioprocess Solutions,1,,
7,Sep 14 2022 6:45PM,246470,16,Sartorius Lab Products and Service,1,,
8,Sep 14 2022 6:01PM,246468,16,Sartorius Bioprocess Solutions,1,,
9,Sep 14 2022 4:58PM,246466,20,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1,,
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1,,
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc.",1,,
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd",6,,
4,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1.0,NaN,NaN
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
4,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1.0,0.0,0.0
2,Sep 15 2022 8:30AM,246469,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Sep 15 2022 7:55AM,246475,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
4,Sep 15 2022 7:34AM,246474,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1478606,5.0,1.0,0.0
16,739409,3.0,0.0,0.0
19,985782,8.0,3.0,0.0
20,985813,12.0,16.0,6.0
22,492947,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1478606,5.0,1.0,0.0
1,16,739409,3.0,0.0,0.0
2,19,985782,8.0,3.0,0.0
3,20,985813,12.0,16.0,6.0
4,22,492947,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,5.0,1.0,0.0
1,16,3.0,0.0,0.0
2,19,8.0,3.0,0.0
3,20,12.0,16.0,6.0
4,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,5.0
1,16,Released,3.0
2,19,Released,8.0
3,20,Released,12.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Completed,0.0,0.0,0.0,6.0,0.0
Executing,1.0,0.0,3.0,16.0,1.0
Released,5.0,3.0,8.0,12.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0
1,Executing,1.0,0.0,3.0,16.0,1.0
2,Released,5.0,3.0,8.0,12.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0
1,Executing,1.0,0.0,3.0,16.0,1.0
2,Released,5.0,3.0,8.0,12.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()